Student ID: 3034963465

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from tqdm import tqdm
import skimage
import skimage.data
from skimage.color import rgb2gray
from skimage.filters import threshold_mean
from skimage.transform import resize
import math


In [ ]:
import sys
!{sys.executable} -m pip install mat73
import mat73


In [ ]:
import sys
!{sys.executable} -m pip install pynverse
import pynverse
from pynverse import inversefunc


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
datadir = "/content/gdrive/MyDrive/NeuralComputation/"
dataset = mat73.loadmat(datadir+'PS4_patterns.mat')
patterns = dataset['patterns']


In [ ]:
figure,axes = plt.subplots(1,3, figsize = (8,8))
axes[0].imshow(patterns[0])
axes[1].imshow(patterns[1])
axes[2].imshow(patterns[2])


In [ ]:
pats = np.zeros((patterns.shape[0],patterns.shape[1]*patterns.shape[2]))
for i in range(patterns.shape[0]):
  pats[i]=patterns[i].flatten()

# **Part 1: Hopfield Network**

i) 

In [ ]:
def store_img_in_weights(X):
    print("Store images in weight matrix")
    n_data =  len(X)
    n_neuron = X[0].shape[0]
        
    # initialize weights
    weights = np.zeros((n_neuron, n_neuron))
        
    # apply hebbian learning
    for i in tqdm(range(n_data)):
        weights += np.outer(X[i], X[i])
        
    # diagonal needs to be 0
    diagonalW = np.diag(np.diag(weights))
    weights = weights - diagonalW
        
    return weights,n_neuron

# energy function -- used to determine when to stop training!
def energy(s,W):
  return -0.5 * s @ W @ s 


def update_state(initial_state, weights, iterations, n_neurons):
  state = initial_state
  e = energy(state,weights)
  # iterate
  store_iteration_states =[]
  for i in range(iterations):
    for j in range(100):
      # start with some random neuron;
      neuron_index = np.random.randint(0, n_neurons) 
      # state update
      state[neuron_index] = np.sign(weights[neuron_index].T @ state)
      store_iteration_states.append(state)
    # Compute new state energy
    e_iter = energy(state,weights)
    if i == iterations/2:
      midway_pattern = state    
    # if equillibrium - return state
    if e == e_iter:
        midway_pattern = store_iteration_states[int(np.round(i/2))]
        return state,midway_pattern
    # if not equillibrium update energy and continue iterating 
    e = e_iter


  return state, midway_pattern


def predict_pattern(X, weights, n_iters, n_neuron):
  X_copy = np.copy(X)
  predicted_patterns = []
  midway_patterns_all = []
  for i in tqdm(range(len(X))):
      final_patterns, midway_pattern = update_state(X_copy[i],weights,n_iters,n_neuron)
      predicted_patterns.append(final_patterns)
      midway_patterns_all.append(midway_pattern)

  return predicted_patterns,midway_patterns_all



In [ ]:
def corrupt(img, pct_corrupt):
    corrupted_img = np.copy(img)
    inv = np.random.binomial(n=1, p=pct_corrupt, size=len(img))
    for i, v in enumerate(img):
        if inv[i]:
            corrupted_img[i] = -1 * v
    return corrupted_img



def plot_results(X, corrupted_pattern, predicted_pattern, figsize=(5, 6)):
    X = [d.reshape(10,10) for d in X]
    corrupted_pattern = [d.reshape(10,10) for d in corrupted_pattern]
    predicted_pattern = [d.reshape(10,10) for d in predicted_pattern]

    fig, axes = plt.subplots(len(X), 3, figsize=figsize)
    for i in range(len(X)):
        if i==0:
            # set titles for each column in the plot
            axes[i, 0].set_title('Data patterns')
            axes[i, 1].set_title("Corrupted patterns")
            axes[i, 2].set_title('Predicted patterns')

        axes[i, 0].imshow(X[i])
        axes[i, 0].axis('off')
        axes[i, 1].imshow(corrupted_pattern[i])
        axes[i, 1].axis('off')
        axes[i, 2].imshow(predicted_pattern[i])
        axes[i, 2].axis('off')

    plt.show()


weights,n_neuron  = store_img_in_weights(pats)
corrupted_pats = [corrupt(d, 0.50) for d in list(pats)]
num_iter=50
predicted_pats,midway_patterns_all = predict_pattern(corrupted_pats, weights, num_iter, n_neuron)


In [ ]:
plot_results(list(pats), corrupted_pats, predicted_pats)

In [ ]:
figure,axes = plt.subplots(1,3, figsize = (6,4))
figure.suptitle('Initial')
axes[0].imshow(corrupted_pats[0].reshape(10,10))
axes[1].imshow(corrupted_pats[1].reshape(10,10))
axes[2].imshow(corrupted_pats[2].reshape(10,10))


In [ ]:
figure,axes = plt.subplots(1,3, figsize = (6,4))
figure.suptitle('Midway training')

axes[0].imshow(midway_patterns_all[0].reshape(10,10))
axes[1].imshow(midway_patterns_all[1].reshape(10,10))
axes[2].imshow(midway_patterns_all[2].reshape(10,10))


In [ ]:
figure,axes = plt.subplots(1,3, figsize = (6,4))
figure.suptitle('Final training', fontsize=16)

axes[0].imshow(predicted_pats[0].reshape(10,10))
axes[1].imshow(predicted_pats[1].reshape(10,10))
axes[2].imshow(predicted_pats[2].reshape(10,10))


ii)

In [ ]:
def create_random_patterns(number_of_patterns,number_of_bits_per_pattern, p_bit):
    return 2 * np.random.binomial(1, p_bit, size=(number_of_patterns, number_of_bits_per_pattern)) - 1


def compare_initial_and_converged_states(init_state, converged_state,dim):
  
    init_state = init_state/dim 
    converged_state = converged_state/dim 
    check_converge = abs(converged_state @ init_state.T).max(axis=1)
    n_good = np.isclose(check_converge, np.ones_like(check_converge)).sum()

    return converged_state.shape[0], n_good


number_of_patterns = [1,2,3,4,5,6,7,8,9,10]
number_of_bits_per_pattern = 100
p_bit = 0.3

iterations=10
all_pats_corr = []
for i in range(len(number_of_patterns)):
  pct_corr=[]
  for n in range(iterations):
    
    generated_patterns = create_random_patterns(number_of_patterns[i],number_of_bits_per_pattern, p_bit)
    weights,n_neuron  = store_img_in_weights(generated_patterns)
    predicted_pats_generated,midway_patterns_all_generated = predict_pattern(list(generated_patterns), weights, num_iter, n_neuron)

    number_initial, number_good = compare_initial_and_converged_states(np.atleast_2d(generated_patterns).astype(float), np.atleast_2d(predicted_pats_generated).astype(float),np.sqrt(number_of_bits_per_pattern))
    pct_corr.append(number_good/number_initial)
  all_pats_corr.append(np.mean(pct_corr))
  
    


In [ ]:
all_pats_corr

In [ ]:
figure,ax = plt.subplots(1,1,figsize=(6,6))
plt.plot(np.arange(0,len(number_of_patterns)),1-np.array(all_pats_corr))
plt.axhline(y=0.5, color='r', linestyle='-')
plt.xlabel('number of patterns')
plt.ylabel('% errors')
ax.set_xticks(number_of_patterns)


In [ ]:
len(np.arange(0,2*math.pi,0.063))

# **Part 2: Ring Attractor Network**

In [ ]:
# tuning curve 
A = 1
K = 8 
B = (40-1)/np.exp(K)
thetas = np.arange(0,2*math.pi,0.063)
theta0 = math.pi
f = []

def tuning_curve_fun(A,B,K,theta0,theta):
  angleDiff = np.cos(theta0-theta)
  return A+B*(np.exp(angleDiff*K))


for i,theta in enumerate(thetas):
  f.append(tuning_curve_fun(A,B,K,theta0,theta))

f = np.array(f)
figure = plt.figure(figsize=(6,6))
plt.plot(thetas-3.14,f)
plt.xlabel('theta - theta_0')
plt.ylabel('firing rate')

In [ ]:
# sigmoid
beta = 0.8
b = 10
c = 0.5
a = 6.34
theta0 = math.pi
xx= np.array(f)

def sigma_fun(x):
  var = 1+np.exp(b*(x+c))
  return a*(np.log(var)**beta)

theta_f = sigma_fun(thetas-theta0)
# plt.plot(thetas-theta0,theta_f)


inverse_sigma_fun=inversefunc(sigma_fun)

fig,axes = plt.subplots(1,2,figsize=(6,4))
axes[0].plot(thetas-theta0,theta_f)
axes[0].set_title('sigma')
axes[1].plot(thetas-theta0,inverse_sigma_fun(thetas-theta0))
axes[1].set_title('inverse_sigma')


In [ ]:
fig = plt.figure(figsize=(6,6))
plt.plot(f,'r',label="f")
plt.plot(sigma_fun(inverse_sigma_fun(f)),'b--',label="sigma(sigma^-1(f))")
plt.legend(loc="upper left")

In [ ]:
x = thetas-theta0
f_fft = np.fft.fft(f)
u_fft = np.fft.fft(inverse_sigma_fun(f))
lambdas = np.array([10e-1,10e-2,10e-3,10e-4,10e-5])


fig,axes = plt.subplots(5,1,figsize=(8,12))
fig.tight_layout(pad=5.0)
w_lambdaZ = []
for i in range(len(lambdas)):
  w_fft = (u_fft * f_fft)/((lambdas[i]*np.max(np.abs(f_fft))**2)+(np.abs(f_fft))**2)
  
  w = np.fft.ifft(w_fft)
  w_lambdaZ.append(np.fft.fftshift(w))
  axes[i].plot(x,np.fft.fftshift(w))
  axes[i].set_title(str(lambdas[i])+'*max(abs(f_ft)^2)')
  axes[i].set_xticklabels([0,-180,-90,-45,0,45,90,180])


ii)

In [ ]:

delta_t_over_tau = .01
num_iterations = 10000
wlambdaZii = w_lambdaZ[2].copy()
def sigma_fun(x):
  var = 1+np.exp(b*(x+c))
  return a*(np.log(var)**beta)

u = np.random.normal(0,1,len(w))
blah = []
for i in range(num_iterations):

  f_fft = np.fft.fft(sigma_fun(u))
  w_fft = np.fft.fft(wlambdaZii)

  u_conv_w = np.fft.ifft(f_fft * w_fft)
  u_conv_w = np.fft.fftshift(u_conv_w)
  u = ((1-delta_t_over_tau)*u) + (delta_t_over_tau*u_conv_w)
  blah.append(u)





In [ ]:
figure,axes = plt.subplots(8,1,figsize=(6,12))
for i in np.arange(1,9):
  axes[i-1].plot(x,sigma_fun(blah[i*1000]))
  axes[i-1].set_ylim([0,40])
  axes[i-1].set_xticklabels([0,0,90,135,180,225,270,360])




In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib


In [ ]:
!{sys.executable} -m pip install ffmpeg
import ffmpeg


In [ ]:
blah[0].shape

In [ ]:
import warnings
warnings.filterwarnings("ignore")

%matplotlib notebook

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()
fig, ax = plt.subplots()
ax.set_ylim([0,40])

def getImageFromList(x):
  
    return sigma_fun(blah[x])

ims=[]
for i in np.arange(1000,5000,50):
    im ,= plt.plot(x,getImageFromList(i),'b',animated=True)
    ims.append([im])

ani = matplotlib.animation.ArtistAnimation(fig, ims, interval=10)
plt.title('plot part 2ii')

plt.close()


# Show the animation
HTML(ani.to_html5_video())

iii)

In [ ]:
%matplotlib inline



# plt.plot(wlambdaZiii)


In [ ]:
gamma = -.001
u = np.random.normal(0,1,len(w))
wlambdaZiii = w_lambdaZ[2].copy() 
iterations = 10000
blahderivative=[]
for i in range(iterations):

  gradientw = np.gradient(wlambdaZiii)
  wlambdaZiii += gradientw*gamma

  f_fft = np.fft.fft(sigma_fun(u))
  w_fft = np.fft.fft(wlambdaZiii)

  u_conv_w = np.fft.ifft(f_fft * w_fft)
  u_conv_w = np.fft.fftshift(u_conv_w)
  u = ((1-delta_t_over_tau)*u) + (delta_t_over_tau*u_conv_w)

  blahderivative.append(u)

In [ ]:

# plt.plot(sigma_fun(blahderivative[9000]))
warnings.filterwarnings("ignore")

%matplotlib notebook

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()
fig, ax = plt.subplots()
ax.set_ylim([0,40])

def getImageFromList(x):
  
    return sigma_fun(blahderivative[x])

ims=[]
for i in np.arange(2000,4000,10):
    im ,= plt.plot(x,getImageFromList(i),'b',animated=True)
    ims.append([im])

ani = matplotlib.animation.ArtistAnimation(fig, ims, interval=10)
plt.title('plot part 2iii')

plt.close()


# Show the animation
HTML(ani.to_html5_video())

In [ ]:
%matplotlib inline
figure,ax = plt.subplots(1,1,figsize=(6,6))


plt.plot(x,sigma_fun(blahderivative[3000]),label='t')
plt.plot(x,sigma_fun(blahderivative[3100]),label='t+10')
plt.plot(x,sigma_fun(blahderivative[3200]),label='t+20')
plt.plot(x,sigma_fun(blahderivative[3300]),label='t+30')
ax.set_xticklabels([0,0,90,135,180,225,270,360])

plt.legend(loc = "upper right")